In [1]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

from xxchallenge import *


# if using fixed position rod,
# cannot optimize G0 only because there is no gradient computed
# because of abs pos


# Parameters
A = -1.62
G0 = tlm.parameter(800.)
T = 0
R = 0


cylinder = tlm.ImplicitCylinder(*torch.tensor([-50/2, 50/2, 37.02/2], dtype=torch.float64).unbind())
rod_data = StoreVar(lambda data: data)

# Primary mirror
primary = tlm.Parabola(1800, A=A, normalize=True)
primary_data = StoreVar(lambda data: data)

# Optical model
optics = tlm.Sequential(
    tlm.Gap(-1000),
    XXLightSource.load(),

    # Primary mirror
    tlm.Gap(G0),
    tlm.Translate3D(y=T),
    tlm.Rotate3D(z=R),
    primary_data,
    tlm.ReflectiveSurface(primary),

    # Fixed rod
    tlm.AbsolutePosition(x=-250),
    rod_data,
    NonImagingRod(cylinder),
)

xxrender(optics, sampling={"xx": 500, "letter": "both"})

Non imaging rod got 500 rays


In [3]:
print(optics)
print(list(optics.named_parameters()))

param_groups = [
    #{'params': [A], 'lr': 1e-4},
    {'params': [G0], 'lr': 50},
    #{'params': [T], 'lr': 50},
    #{'params': [R], 'lr': 1},
]

record = tlm.optimize(
    optics,
    optimizer = optim.SGD(param_groups),
    sampling = {"xx": 60, "disable_viewer": True, "letter": "negative"},
    dim = 3,
    num_iter = 100
)

print()
print("Final values")
print("A", A.item())
print("G0", G0.item())
print("T", T.item())
print("R", R.item())

# Print rod position
target = rod_data.value.target()
print("ROD X", target[1].item())
print("ROD Y", target[2].item())
print("ROD Z", -target[0].item())

plot_record(record, param_groups, optics)
record.best()

xxrender(optics, sampling={"xx": 500, "letter": "both"})

Sequential(
  (0): Gap()
  (1): XXLightSource()
  (2): Gap()
  (3): Translate3D()
  (4): Rotate3D()
  (5): StoreVar()
  (6): ReflectiveSurface(
    (collision_surface): CollisionSurface()
  )
  (7): AbsolutePosition()
  (8): StoreVar()
  (9): NonImagingRod()
)
[('2.offset', Parameter containing:
tensor(800., dtype=torch.float64, requires_grad=True))]
Non imaging rod got 60 rays


RuntimeError: No differentiable loss computed by optical stack (loss.requires_grad is False)